In [3]:
import pandas as pd
from sys import *
from subprocess import *
import os

In [4]:
mobi_data = pd.read_csv("../../data/Mobi_System_Data_2020.csv", compression='zip').dropna()
geo_data = pd.read_csv('../../data/geocodings.csv',index_col=0)
data = mobi_data.merge(geo_data.rename({'lat':'Departure lat','long':'Departure long'},axis=1),left_on='Departure station',right_on='address').drop('address',axis=1)
data = data.merge(geo_data.rename({'lat':'Return lat','long':'Return long'},axis=1),left_on='Return station',right_on='address').drop('address',axis=1)
print(data.columns)
data["time"] = pd.to_datetime(data["Departure"], format="%Y-%m-%d %H:%M:%S")
data["hour"] = data["time"].dt.hour
data["day"] = data["time"].dt.day
data["month"] = data["time"].dt.month
data["year"] = data["time"].dt.year


Index(['Unnamed: 0', 'Departure', 'Return', 'Bike', 'Departure station',
       'Return station', 'Membership type', 'Covered distance (m)',
       'Duration (sec.)', 'Departure battery voltage (mV)',
       'Return battery voltage (mV)', 'Departure temperature (C)',
       'Return temperature (C)', 'Stopover duration (sec.)',
       'Number of stopovers', 'postal_code_x', 'Departure lat',
       'Departure long', 'postal_code_y', 'Return lat', 'Return long'],
      dtype='object')


In [5]:
stations = data[["Departure lat", "Departure long", "Departure station", "postal_code_x"]].drop_duplicates()
stations = stations.rename(columns={"Departure lat": "lat", "Departure long": "long", "Departure station": "station", "postal_code_x": "postal_code"})
stations = stations.loc[stations["postal_code"].str.startswith("V")]
stations["postal_code"] = stations["postal_code"].str[0:3]

In [6]:
def min_max_norm(series):
    min = series.min()
    max = series.max()
    return (series - min) / (max - min)

In [7]:
radius_parks = 500
radius_stops = 500
radius_bikeways = 500
os.chdir("../data_server")
for s in stations.iterrows():
    lat, long = s[1][0], s[1][1]
    stations.loc[s[0], "parks"] = float(check_output(["python", "data_processor.py", "parks", str(long), str(lat), str(radius_parks)]))
    # for p in parks.iterrows():
    #     p_lat, p_long = p[1][0], p[1][1]
    #     distance = haversine(lat, long, p_lat, p_long)
    #     if distance < 0.5:
    #         stations.loc[s[0], "parks"] = 1
    #         break
    stations.loc[s[0], "stops"] = float(check_output(["python", "data_processor.py", "translink_stops", str(long), str(lat), str(radius_stops)]))
    # for t in translink_stops.iterrows():
    #     t_lat, t_long = t[1][1], t[1][2]
    #     distance = haversine(lat, long, t_lat, t_long)
    #     if distance < 0.25:
    #         stations.loc[s[0], "stops"] += 1
    stations.loc[s[0], "bike_ways"] = float(check_output(["python", "data_processor.py", "translink_stops", str(long), str(lat), str(radius_bikeways)]))
    postal_code = s[1][3]
    p = check_output(["python", "data_processor.py", "population", postal_code])
    stations.loc[s[0], "population"] = 0 if p == b"Not found\r\n" else float(p)
    # search = population.loc[population["Geographic code"] == postal_code]
    # if search.shape[0] > 0:
    #     stations.loc[s[0], "population"] = search.iloc[0, 2]
    # else:
    #     stations.loc[s[0], "population"] = 0
max_p, min_p = stations["bike_ways"].max(), stations["bike_ways"].min()
stations["parks"] = min_max_norm(stations["parks"])
stations["stops"] = min_max_norm(stations["stops"])
stations["bike_ways"] = min_max_norm(stations["bike_ways"])
stations["population"] = min_max_norm(stations["population"])
os.chdir("../")
stations.to_csv("../data/stations.csv")